In [2]:
import os
from dotenv import load_dotenv
from pathlib import Path

# Construct path two directories up from the **current working directory**
env_path = Path.cwd().resolve().parents[1] / '.env'

# Load .env from that path
load_dotenv(dotenv_path=env_path)

cohere_api_key = os.getenv('COHERE_API_KEY')
if cohere_api_key is None:
    raise ValueError("COHERE_API_KEY not set in .env")

print(cohere_api_key)  # For debug only

groq_api_key = os.getenv('GROQ_API_KEY')
if groq_api_key is None:
    raise ValueError("GROQ_API_KEY not set in .env")

# Optional: explicitly set in os.environ if required by libraries
os.environ['COHERE_API_KEY'] = cohere_api_key
os.environ['GROQ_API_KEY'] = groq_api_key


ul01XOnxP4BW2SYjirDI0s1BcCXjHE7mNvCpmqkI


In [4]:
from groq import Groq

client = Groq()

chat_completion = client.chat.completions.create(
    messages=[
        # Set an optional system message. This sets the behavior of the
        # assistant and can be used to provide specific instructions for
        # how it should behave throughout the conversation.
        {
            "role": "system",
            "content": "You are a doctor, explain things thoroughly, try to analyze which illness could the patient suffer from. and what's the best solution to resolve it."
        },
        # Set a user message for the assistant to respond to.
        {
            "role": "user",
            "content": "Teach me how to learn quicker",
        }
    ],

    # The language model which will generate the completion.
    model="llama-3.3-70b-versatile"
)

print("Given 3 choices with higher randomness: \n")
num_choices = 3
#for i in range(num_choices):
 #   print(chat_completion.choices[i].message.content)

# Print the completion returned by the LLM.
print(chat_completion.choices[0].message.content)

Given 3 choices with higher randomness: 

Learning how to learn quicker is a valuable skill that can benefit you in many areas of life. Here are some effective strategies to help you learn quicker:

1. **Set Clear Goals**: Before starting to learn, define what you want to achieve. Setting specific, measurable, and attainable goals will help you stay focused and motivated.
2. **Use Active Recall**: Instead of just re-reading your notes, actively try to recall the information from memory. Quiz yourself, test your knowledge, and try to summarize what you've learned in your own words.
3. **Spaced Repetition**: Review material at increasingly longer intervals to help solidify it in your long-term memory. This technique can help you retain information more effectively.
4. **Chunking**: Break down complex information into smaller, manageable chunks. Organize these chunks into categories, create concept maps, or use mind maps to visualize relationships between ideas.
5. **Mnemonics**: Use asso

In [7]:
type(chat_completion.choices[0])

groq.types.chat.chat_completion.Choice

In [5]:
import cohere

co = cohere.ClientV2()

text_inputs = [
    {
        "content": [
            {"type": "text", "text": "hello"},
            {"type": "text", "text": "goodbye"}
        ]
    },
]

response = co.embed(
    inputs=text_inputs,
    model="embed-v4.0",
    input_type="classification",
    embedding_types=["float"],
)
print(response)


id='82802a8b-1587-415d-a444-7b5751c9110e' embeddings=EmbedByTypeResponseEmbeddings(float_=[[0.026611328, -0.04321289, 0.007293701, -0.0051574707, 0.017456055, 0.002319336, -0.036376953, 0.02709961, -0.033203125, 0.010925293, -0.008056641, 0.055419922, -0.04711914, -0.04736328, 0.009155273, 0.010681152, -0.008300781, 0.041992188, -0.0008735657, -0.032470703, -0.025512695, 0.024414062, 0.00340271, -0.037109375, -0.021118164, -0.0034637451, 0.021606445, 0.005706787, 0.034179688, -0.024414062, -0.033691406, 0.014038086, 0.008605957, 0.015258789, -0.025268555, 0.0050964355, -0.017700195, -0.016723633, 0.040283203, 0.015014648, -0.009277344, 0.008056641, 0.0038452148, -0.005218506, 0.022338867, 0.026855469, 0.010803223, -0.016723633, -0.0011901855, -0.0079956055, -0.0014724731, 0.01586914, -0.013366699, 0.0020141602, 0.03881836, 0.00019264221, 0.022094727, 0.056396484, -0.005554199, -0.0019378662, -0.013183594, 0.020751953, -0.013366699, -0.0023040771, 0.0023040771, 0.029907227, -0.052246094

In [ ]:
# Reranking the embeddings
co_rerank = cohere.ClientV2()

docs = [
    "Carson City is the capital city of the American state of Nevada.",
    "The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean. Its capital is Saipan.",
    "Capitalization or capitalisation in English grammar is the use of a capital letter at the start of a word. English usage varies from capitalization in other languages.",
    "Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district.",
    "Capital punishment has existed in the United States since beforethe United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states.",
]

response = co_rerank.rerank(
    model="rerank-v3.5",
    query="What is the capital of the United States?",
    documents=docs,
    top_n=3,
)
print(response)


In [ ]:
import os

# Starting from your current working directory
base_dir = os.getcwd()  # or set explicitly

user_id = 1 # In future I should get this via API

# Build the path
media_path = os.path.join(base_dir, "media", "documents", f"user_{user_id}")

print("Full path:", media_path)


In [7]:
"""
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS

urls = [
    "https://www.matthewlozinski.com/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

vectorstore = FAISS.from_documents(
    documents=doc_splits,
    embedding=co,
)

vectorstore_retriever = vectorstore.as_retriever()
"""

AttributeError: 'ClientV2' object has no attribute 'embed_documents'

In [ ]:
import os

# Path to shared documents inside the container
documents_path = "/app/shared_documents/user_1"

# List files
files = os.listdir(documents_path)

# Print the list of files
print(files)